# ADAPTING PROCESSED DATA TO FIT INTO MODEL

In [12]:
# Import packages
import pandas as pd
import numpy as np
import ndjson

# from utils import upload_blobS

ModuleNotFoundError: No module named 'code.utils'; 'code' is not a package

In [ ]:
# Download information from GCloud buckets

In [10]:
# Applying padding to a drawing array
def add_padding(array_drawing: np.ndarray, max_length: int) -> np.ndarray:
    
    # Define values for padding layers - e.g. [99,99,99]
    padding = [[99,99,99]]
    
    # If array is greater than max_length slice off remainder of array
    if len(array_drawing) >= max_length : 
        return array_drawing[0:150]
    
    # If array is less than max_length, adding padding 
    else :
        pad_length = max_length - len(array_drawing)
        return np.concatenate((array_drawing,np.array(padding*pad_length)),axis=0)

In [ ]:
# 

In [13]:
# Upload a file to a bucket
def upload_blob(bucket_name, source_file_name, destination_blob_name) -> None:
    '''
    Uploads a file to the bucket.
    '''
    # Initialize a client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # Create a blob
    blob = bucket.blob(destination_blob_name)

    # Upload the file to the blob
    blob.upload_from_filename(source_file_name)